In [1]:
from fastapi import FastAPI, HTTPException
import pandas as pd
from fastapi.responses import JSONResponse
from fastapi.encoders import jsonable_encoder

app = FastAPI()

# Cargar el dataframe
try:
    Steam_games_important = pd.read_parquet("C:/Users/crisr/OneDrive/Escritorio/Mateo/Programacion/SoyHenry/Proyecto_Individual_uno/carpeta_raiz/Datasets/Steam_games_endpoint_1.parquet")
except Exception as e:
    print(f"Error al cargar el archivo parquet: {e}")

@app.get("/")
def index():
    return {"Hello": "World"}

@app.get("/developer/{desarrollador}")
def developer(desarrollador: str):
    try: 
        salida = {}

        if desarrollador not in Steam_games_important["developer"].unique():
            raise HTTPException(status_code=404, detail="Developer not found")
        
        for i, año in enumerate(Steam_games_important[Steam_games_important["developer"] == desarrollador]["release_year"].unique()):

            cant_items_anual = Steam_games_important[(Steam_games_important["release_year"] == año) & (Steam_games_important["developer"] == desarrollador)]["developer"].count()

            cant_items_gratuitos_anual = Steam_games_important[(Steam_games_important["release_year"] == año) & (Steam_games_important["developer"] == desarrollador) & (Steam_games_important["price"] == 0)]["developer"].count()
            
            pre_salida = {
                "Año": año, 
                "Cantidad items": cant_items_anual, 
                "Contenido gratuito (%)": f"{(cant_items_gratuitos_anual / cant_items_anual) * 100}%"
            }
            
            salida[i] = pre_salida


        return salida
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Internal Server Error: {e}")




In [2]:
developer("ebi-hime")

{0: {'Año': 2015, 'Cantidad items': 1, 'Contenido gratuito (%)': '0.0%'},
 1: {'Año': 2016, 'Cantidad items': 5, 'Contenido gratuito (%)': '20.0%'},
 2: {'Año': 2017, 'Cantidad items': 5, 'Contenido gratuito (%)': '40.0%'}}

In [13]:
def developer(desarrollador: str):

        salida = {}

        if desarrollador not in Steam_games_important["developer"].unique():
            raise HTTPException(status_code=404, detail="Developer not found")
        
        for año in Steam_games_important[Steam_games_important["developer"] == desarrollador]["release_year"].unique():

            cant_items_anual = Steam_games_important[(Steam_games_important["release_year"] == año) & (Steam_games_important["developer"] == desarrollador)]["developer"].count()

            cant_items_gratuitos_anual = Steam_games_important[(Steam_games_important["release_year"] == año) & (Steam_games_important["developer"] == desarrollador) & (Steam_games_important["price"] == 0)]["developer"].count()
            
            pre_salida = {
                "Cantidad items": cant_items_anual, 
                "Contenido gratuito (%)": f"{(cant_items_gratuitos_anual / cant_items_anual) * 100}%"
            }
            
            salida[año] = pre_salida


        return salida

In [14]:
developer("ebi-hime")

{2015: {'Cantidad items': 1, 'Contenido gratuito (%)': '0.0%'},
 2016: {'Cantidad items': 5, 'Contenido gratuito (%)': '20.0%'},
 2017: {'Cantidad items': 5, 'Contenido gratuito (%)': '40.0%'}}

In [7]:
Steam_games_important["release_year"].unique()

array([2018, 2017, 1997, 1998, 2016, 2006, 2005, 2003, 2007, 2002, 2000,
       1995, 1996, 1994, 2001, 1993, 2004, 2008, 2009, 1999, 1992, 1989,
       2010, 2011, 2013, 2012, 2014, 1983, 1984, 2015, 1990, 1988, 1991,
       1987, 1986, 2021, 2019, 1985])

In [8]:
Steam_games_important[Steam_games_important["developer"] =="ebi-hime"]["release_year"].unique()

array([2015, 2016, 2017])

In [10]:
enumerate(Steam_games_important[Steam_games_important["developer"] == "ebi-hime"]["release_year"].unique())

In [3]:
@app.get("/developer/{desarrollador}",response_model=dict)
async def developer_use(desarrollador_local : str):
    try:
        Steam_games_important = pd.read_parquet("C:/Users/crisr/OneDrive/Escritorio/Mateo/Programacion/SoyHenry/Proyecto_Individual_uno/carpeta_raiz/Datasets/Steam_games_endpoint_1.parquet")
        
        resultado = developer(desarrollador_local)

        return JSONResponse(content=jsonable_encoder(resultado), media_type="application/json")
    except FileNotFoundError:
        raise HTTPException(status_code=404, detail="Archivo Parquet no encontrado, revisa si la ruta del archivo es correcta ;)")
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Error al leer el archivo Parquet: {str(e)}")             